### In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.

In [5]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [6]:
#Create a table rentals_may to store the data from rental table with information for the month of May.

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

rentals_may = pd.read_sql_query('SELECT * FROM rental WHERE rental_date LIKE "2005-05%%"', engine)
rentals_may.tail()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53
1155,1157,2005-05-31 22:47:45,1449,61,2005-06-02 18:01:45,1,2006-02-15 21:30:53


In [8]:
#Insert values in the table rentals_may using the table rental, filtering values only for the month of May.
query = '''
       insert into rental(rental_id, rental_date, inventory_id, customer_id, return_date, staff_id)
       values(16054,'2005-05-31 22:48:45',1450,61,'2005-05-02 18:01:45',2);
      '''
engine.execute(query)

In [13]:
#Create a table rentals_june to store the data from rental table with information for the month of June.
#pd.to_sql
#table.to_sql
query = '''
        CREATE TABLE `rentals_june` (
            `rental_id` int NOT NULL AUTO_INCREMENT,
            `rental_date` datetime NOT NULL,
            `inventory_id` mediumint unsigned NOT NULL,
            `customer_id` smallint unsigned NOT NULL,
            `return_date` datetime DEFAULT NULL,
            `staff_id` tinyint unsigned NOT NULL,
            `last_update` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            PRIMARY KEY (`rental_id`)
            ) ENGINE=InnoDB AUTO_INCREMENT=16055 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;
'''
engine.execute(query)

In [ ]:
#Insert values in the table rentals_june using the table rental, filtering values only for the month of June.
#Insert done with import wizard

Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:
* Check the number of rentals for each customer for May
* Check the number of rentals for each customer for June

Hint: You can store the results from the two queries in two separate dataframes.


In [21]:
#Check the number of rentals for each customer for May.
query = '''
        SELECT customer_id, count(rental_id) AS rentals_may
        FROM sakila.rental
        WHERE rental_date LIKE "20%%-05%%"
        GROUP BY customer_id;
        '''

customers_may = pd.read_sql_query(query, engine)
customers_may.head()

,customer_id,rentals_may
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3


In [22]:
#Check the number of rentals for each customer for June.
query = '''
        SELECT customer_id, count(rental_id) AS rentals_june
        FROM sakila.rental
        WHERE rental_date LIKE "20%%-06%%"
        GROUP BY customer_id;
        '''

customers_june = pd.read_sql_query(query, engine)
customers_june.head()

,customer_id,rentals_june
0,1,7
1,2,1
2,3,4
3,4,6
4,5,6


Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the merge function.

In [ ]:
query = '''
        SELECT customer_id, count(rental_id) AS rentals_june, count(rental_id) AS rentals_may
        FROM sakila.rental
        WHERE rental_date LIKE "20%%-06%%"
        GROUP BY customer_id;
        '''

may_vs_june = pd.read_sql_query(query, engine)
may_vs_june.head()

In [26]:
may_vs_june=customers_may.merge(customers_june, how='right', on='customer_id')
may_vs_june.head()

,customer_id,rentals_may,rentals_june
0,1,2.0,7
1,2,1.0,1
2,3,2.0,4
3,4,NaN,6
4,5,3.0,6


In [33]:
df = may_vs_june
df['rentals_may'].isna().sum()

78

In [35]:
df.dtypes

customer_id       int64
rentals_may     float64
rentals_june      int64
compare          object
dtype: object

In [41]:
df['rentals_may'] = df['rentals_may'].fillna(0)

In [43]:
df.dtypes

customer_id       int64
rentals_may     float64
rentals_june      int64
compare          object
dtype: object

In [ ]:
df['rentals_may'] = df['rentals_may'].fillna(0)

In [50]:
df['rentals_may'] = df['rentals_may'].apply(lambda x: int(x))

In [52]:
def compare(may_vs_june):
    if may_vs_june['rentals_may'] > may_vs_june['rentals_june']:
        return 'More in May'
    elif may_vs_june['rentals_june'] > may_vs_june['rentals_may']:
        return 'More in June'
    elif may_vs_june['rentals_june'] == may_vs_june['rentals_may']:
        return 'Same number of rentals in both moths'
df['compare'] = df.apply(compare, axis=1)
df

,customer_id,rentals_may,rentals_june,compare
0,1,2,7,More in June
1,2,1,1,Same number of rentals in both moths
2,3,2,4,More in June
3,4,0,6,More in June
4,5,3,6,More in June
...,...,...,...,...
586,595,1,3,More in June
587,596,6,2,More in May
588,597,2,3,More in June
589,598,0,4,More in June
